In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

%matplotlib inline

In [2]:
data_dir = '../noaa-runtime/data/'

In [3]:
dst = pd.read_csv(data_dir +"dst_labels.csv")
dst.timedelta = pd.to_timedelta(dst.timedelta)
dst.set_index(["period", "timedelta"], inplace=True)

sunspots = pd.read_csv(data_dir + "sunspots.csv")
sunspots.timedelta = pd.to_timedelta(sunspots.timedelta)
sunspots.set_index(["period", "timedelta"], inplace=True)

solar_wind = pd.read_csv(data_dir + "solar_wind.csv")
solar_wind.timedelta = pd.to_timedelta(solar_wind.timedelta)
solar_wind.set_index(["period", "timedelta"], inplace=True)

In [ ]:
from numpy.random import seed
from tensorflow.random import set_seed

seed(2020)
set_seed(2021)

## Data Preprocessing

In [ ]:
def aggregate_data(feature_df, aggs=["mean", "std"], agg_time="30min"):
    """Aggregates features to the floor of each hour using mean and standard deviation.
    e.g. All values from "11:00:00" to "11:30:00" will be aggregated to "11:00:00".
    """
    # group by the floor of each hour use timedelta index
    agged = feature_df.groupby(
        ["period", feature_df.index.get_level_values(1).floor(agg_time)]
    ).agg(aggs)
    # flatten hierachical column index
    agged.columns = ["_".join(x) for x in agged.columns]
    return agged

# interpolate the data
def interpolate_data(df):
    """
    # interpolate the data
    - `smoothed_ssn`: forward fill
    - `solar_wind`: interpolation
    """
    df.smoothed_ssn = df.smoothed_ssn.fillna(method="ffill")
    df = df.interpolate()
    return df
    
def preprocess_data(solar_wind, sunspots, scaler=None, subset=None):
    """
    Preprocessing steps:
        - Subset the data
        - Aggregate hourly
        - Join solar wind and sunspot data
        - Scale using standard scaler
        - Impute missing values
    """
    # select features we want to use
    if subset:
        solar_wind = solar_wind[subset]
    
    # aggregate solar wind data and join with sunspots
    data_agg = aggregate_data(solar_wind).join(sunspots)

    # scaling the data
    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(data_agg)
    data_normalized = pd.DataFrame( scaler.transform(data_agg),
                                   index=data_agg.index, columns=data_agg.columns)
    # Interpolate the missing data
    data_interpolated = interpolate_data(data_normalized)
    
    # Return the scaler object as well to use later during prediction
    return data_interpolated, scaler


In [ ]:
SOLAR_WIND_FEATURES = [ "bt", "temperature", "bx_gse", "by_gse",
                       "bz_gse", "speed", "density"]

XCOLS = ( [col + "_mean" for col in SOLAR_WIND_FEATURES]
         + [col + "_std" for col in SOLAR_WIND_FEATURES]
         + ["smoothed_ssn"] )

df_features, scaler = preprocess_data(solar_wind, sunspots, subset=SOLAR_WIND_FEATURES)
print(df_features.shape)
df_features.head()

In [ ]:
# check to make sure missing values are filled
assert (df_features.isna().sum() == 0).all()

In [ ]:
YCOLS = ["t0", "t1"]

def process_labels(dst):
    y = dst.copy()
    y["t1"] = y.groupby("period").dst.shift(-1)
    y.columns = YCOLS
    return y

df_labels = process_labels(dst)
df_labels.head(2)

In [ ]:
df = df_labels.join(df_features)
df.head()

In [ ]:
def get_train_test_val(df, test_per_period, val_per_period):
    """Splits data across periods into train, test, and validation"""
    # assign the last `test_per_period` rows from each period to test
    test = df.groupby("period").tail(test_per_period)
    interim = df[~df.index.isin(test.index)]
    # assign the last `val_per_period` from the remaining rows to validation
    val = interim.groupby("period").tail(val_per_period)
    # the remaining rows are assigned to train
    train = interim[~interim.index.isin(val.index)]
    return train, test, val

df_train, df_test, df_val = get_train_test_val(df, test_per_period=6_000, val_per_period=3_000)

In [ ]:
def plots():
    
    ind = [0, 1, 2]
    names = ["train_a", "train_b", "train_c"]
    width = 0.75
    train_cnts = [len(df) for _, df in df_train.groupby("period")]
    val_cnts = [len(df) for _, df in df_val.groupby("period")]
    test_cnts = [len(df) for _, df in df_test.groupby("period")]

    print (train_cnts, val_cnts, test_cnts)
    
    p1 = plt.barh(ind, train_cnts, width)
    p2 = plt.barh(ind, val_cnts, width, left=train_cnts)
    p3 = plt.barh(ind, test_cnts, width, left=np.add(val_cnts, train_cnts).tolist())

    plt.yticks(ind, names)
    plt.ylabel("Period")
    plt.xlabel("Hourly Timesteps")
    plt.title("Train/Validation/Test Splits over Periods", fontsize=16)
    plt.legend(["Train", "Validation", "Test"])
    
plots()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import preprocessing

In [ ]:
#df[df.index=='train_a'].shape
df[df.index.get_level_values(0)=='train_a'].shape

In [ ]:
for _, dff in df.groupby("period"):
    print (dff[XCOLS].shape)

In [ ]:
data_config = { "timesteps": 32, "batch_size": 32}

def timeseries_dataset_from_df(df, batch_size):
    dataset = None
    timesteps = data_config["timesteps"]

    # iterate through periods
    for _, period_df in df.groupby("period"):
        # realign features and labels so that first sequence of 32 is aligned with the 33rd target
        inputs = period_df[XCOLS][:-timesteps]
        outputs = period_df[YCOLS][timesteps:]

        period_ds = preprocessing.timeseries_dataset_from_array(
            inputs,
            outputs,
            timesteps,
            batch_size=batch_size,
        )

        if dataset is None:
            dataset = period_ds
        else:
            dataset = dataset.concatenate(period_ds)

    return dataset


train_ds = timeseries_dataset_from_df(df_train, data_config["batch_size"])
val_ds = timeseries_dataset_from_df(df_val, data_config["batch_size"])

print(f"Number of train batches: {len(train_ds)}")
print(f"Number of val batches: {len(val_ds)}")

In [ ]:
# define our model
model_config = {"n_epochs": 50, "n_neurons": 32, "dropout": 32, "stateful": False}

def make_model1():
    model = Sequential()
    model.add(
        LSTM(
            model_config["n_neurons"],
            # usually set to (`batch_size`, `sequence_length`, `n_features`)
            # setting the batch size to None allows for variable length batches
            batch_input_shape=(None, data_config["timesteps"], len(XCOLS)),
            stateful=model_config["stateful"],
            dropout=model_config["dropout"],
        )
    )

    model.add(Dense(64))
    model.add(Dense(32))
    
    model.add(Dense(len(YCOLS)))
    model.compile( loss="mean_squared_error", optimizer="adam")
    return model

model = make_model1()
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    batch_size=data_config["batch_size"],
    epochs=model_config["n_epochs"],
    verbose=1,
    shuffle=False,
    validation_data=val_ds,
)

In [ ]:
for name, values in history.history.items():
    plt.plot(values)

In [ ]:
model_config = {"n_epochs": 10, "n_neurons": 64, "dropout": 0.4, "stateful": False}

model.summary()

In [ ]:
print ('n_epochs=',model_config["n_epochs"])

In [ ]:
history = model.fit(
    train_ds,
    batch_size=data_config["batch_size"],
    epochs=model_config["n_epochs"],
    verbose=1,
    shuffle=False,
    validation_data=val_ds,
)

In [ ]:
#for name, values in history.history.items():
#    plt.plot(values)
#loss=
loss=history.history['loss']
val_loss=history.history['val_loss']
plt.plot(range(1, len(loss)+1), loss, label='Loss' )
plt.plot(range(1, len(loss)+1), val_loss, label='Val Loss' )
plt.legend()
plt.show()

In [ ]:

# define our model
model_config = {"n_epochs": 50, "n_neurons": 32, "dropout": 0.4, "stateful": False}

def make_model2():
    model = Sequential()
    model.add(
        LSTM(
            model_config["n_neurons"],
            # usually set to (`batch_size`, `sequence_length`, `n_features`)
            # setting the batch size to None allows for variable length batches
            batch_input_shape=(None, data_config["timesteps"], len(XCOLS)),
            stateful=model_config["stateful"],
            dropout=model_config["dropout"],
        )
    )
    model.add(Dense(32))
    model.add(Dense(32))
    model.add(Dense(32))
    model.add(Dense(len(YCOLS)))
    model.compile(
        loss="mean_squared_error",
        optimizer="adam",
    )
    return model

model = make_model2()
model.summary()

In [ ]:
# First Run
history = model.fit(
    train_ds,
    batch_size=data_config["batch_size"],
    epochs=model_config["n_epochs"],
    verbose=1,
    shuffle=False,
    validation_data=val_ds,
)

In [ ]:
#for name, values in history.history.items():
#    plt.plot(values)
loss=history.history['loss']
val_loss=history.history['val_loss']
plt.plot(range(1, len(loss)+1), loss, label='Loss' )
plt.plot(range(1, len(loss)+1), val_loss, label='Val Loss' )
[plt.axvline(x=i, color='k', lw=.5, ls='--') for i in range(2, 10, 2)]
plt.legend()
plt.show() 

In [ ]:
# Rerun for a smaller epoch
model_config = {"n_epochs": 8, "n_neurons": 64, "dropout": 0.4, "stateful": False}
model.summary()

In [ ]:
# First Run
history = model.fit(
    train_ds,
    batch_size=data_config["batch_size"],
    epochs=model_config["n_epochs"],
    verbose=1,
    shuffle=False,
    validation_data=val_ds,
)

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
plt.plot(range(1, len(loss)+1), loss, label='Loss' )
plt.plot(range(1, len(loss)+1), val_loss, label='Val Loss' )
[plt.axvline(x=i, color='k', lw=.5, ls='--') for i in range(2, 8, 2)]
plt.legend()
plt.show() 

In [ ]:
test_ds = timeseries_dataset_from_df(test, data_config["batch_size"])

In [ ]:
test_ds = timeseries_dataset_from_df(test, data_config["batch_size"])
mse = model.evaluate(test_ds)
print(f"Test RMSE: {mse**.5:.2f}")

In [ ]:
import json
import pickle

model.save("model")

with open("scaler.pck", "wb") as f:
    pickle.dump(scaler, f)

data_config["solar_wind_subset"] = SOLAR_WIND_FEATURES
print(data_config)
with open("config.json", "w") as f:
    json.dump(data_config, f)

In [ ]:

# define our model
model_config = {"n_epochs": 50, "n_neurons": 16, "dropout": 0.25, "stateful": False}

def make_model3():
    model = Sequential()
    model.add(
        LSTM(
            model_config["n_neurons"],
            # usually set to (`batch_size`, `sequence_length`, `n_features`)
            # setting the batch size to None allows for variable length batches
            batch_input_shape=(None, data_config["timesteps"], len(XCOLS)),
            stateful=model_config["stateful"],
            dropout=model_config["dropout"],
        )
    )
    model.add(Dense(16))
    model.add(Dropout(model_config["dropout"]))
    
    model.add(Dense(len(YCOLS)))
    model.compile(
        loss="mean_squared_error",
        optimizer="adam",
    )
    return model

model = make_model3()
model.summary()

In [ ]:
# First Run
history = model.fit(
    train_ds,
    batch_size=data_config["batch_size"],
    epochs=model_config["n_epochs"],
    verbose=1,
    shuffle=False,
    validation_data=val_ds,
)

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
plt.plot(range(1, len(loss)+1), loss, label='Loss' )
plt.plot(range(1, len(loss)+1), val_loss, label='Val Loss' )
[plt.axvline(x=i, color='k', lw=.5, ls='--') for i in range(2, 18, 2)]
plt.legend()
plt.show() 

In [ ]:
# Rerun for a smaller epoch
model_config = {"n_epochs": 17, "n_neurons": 16, "dropout": 0.25, "stateful": False}
model.summary()

In [ ]:
# First Run
history = model.fit(
    train_ds,
    batch_size=data_config["batch_size"],
    epochs=model_config["n_epochs"],
    verbose=1,
    shuffle=False,
    validation_data=val_ds,
)

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
plt.plot(range(1, len(loss)+1), loss, label='Loss' )
plt.plot(range(1, len(loss)+1), val_loss, label='Val Loss' )
[plt.axvline(x=i, color='k', lw=.5, ls='--') for i in range(2, 18, 2)]
plt.legend()
plt.show() 

In [ ]:
test_ds = timeseries_dataset_from_df(test, data_config["batch_size"])
mse = model.evaluate(test_ds)
print(f"Test RMSE: {mse**.5:.2f}")

In [ ]:
import json
import pickle

model.save("model")

with open("scaler.pck", "wb") as f:
    pickle.dump(scaler, f)

data_config["solar_wind_subset"] = SOLAR_WIND_FEATURES
print(data_config)
with open("config.json", "w") as f:
    json.dump(data_config, f)

In [ ]:
import json
import pickle
from typing import Tuple
from tensorflow import keras
import numpy as np
import pandas as pd

# Load in serialized model, config, and scaler
model = keras.models.load_model("model")

with open("config.json", "r") as f:
    CONFIG = json.load(f)

with open("scaler.pck", "rb") as f:
    scaler = pickle.load(f)

# Set global variables    
TIMESTEPS = CONFIG["timesteps"]
SOLAR_WIND_FEATURES = [
    "bt",
    "temperature",
    "bx_gse",
    "by_gse",
    "bz_gse",
    "speed",
    "density",
]
XCOLS = (
    [col + "_mean" for col in SOLAR_WIND_FEATURES]
    + [col + "_std" for col in SOLAR_WIND_FEATURES]
    + ["smoothed_ssn"]
)


# Define functions for preprocessing
def impute_features(feature_df):
    """Imputes data using the following methods:
    - `smoothed_ssn`: forward fill
    - `solar_wind`: interpolation
    """
    # forward fill sunspot data for the rest of the month
    feature_df.smoothed_ssn = feature_df.smoothed_ssn.fillna(method="ffill")
    # interpolate between missing solar wind values
    feature_df = feature_df.interpolate()
    return feature_df


def aggregate_hourly(feature_df, aggs=["mean", "std"]):
    """Aggregates features to the floor of each hour using mean and standard deviation.
    e.g. All values from "11:00:00" to "11:59:00" will be aggregated to "11:00:00".
    """
    # group by the floor of each hour use timedelta index
    agged = feature_df.groupby([feature_df.index.floor("H")]).agg(aggs)

    # flatten hierachical column index
    agged.columns = ["_".join(x) for x in agged.columns]
    return agged


def preprocess_features(solar_wind, sunspots, scaler=None, subset=None):
    """
    Preprocessing steps:
        - Subset the data
        - Aggregate hourly
        - Join solar wind and sunspot data
        - Scale using standard scaler
        - Impute missing values
    """
    # select features we want to use
    if subset:
        solar_wind = solar_wind[subset]

    # aggregate solar wind data and join with sunspots
    hourly_features = aggregate_hourly(solar_wind).join(sunspots)

    # subtract mean and divide by standard deviation
    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(hourly_features)

    normalized = pd.DataFrame(
        scaler.transform(hourly_features),
        index=hourly_features.index,
        columns=hourly_features.columns,
    )

    # impute missing values
    imputed = impute_features(normalized)

    # we want to return the scaler object as well to use later during prediction
    return imputed, scaler


# THIS MUST BE DEFINED FOR YOUR SUBMISSION TO RUN
def predict_dst(
    solar_wind_7d: pd.DataFrame,
    satellite_positions_7d: pd.DataFrame,
    latest_sunspot_number: float,
) -> Tuple[float, float]:
    """
    Take all of the data up until time t-1, and then make predictions for
    times t and t+1.
    Parameters
    ----------
    solar_wind_7d: pd.DataFrame
        The last 7 days of satellite data up until (t - 1) minutes [exclusive of t]
    satellite_positions_7d: pd.DataFrame
        The last 7 days of satellite position data up until the present time [inclusive of t]
    latest_sunspot_number: float
        The latest monthly sunspot number (SSN) to be available
    Returns
    -------
    predictions : Tuple[float, float]
        A tuple of two predictions, for (t and t + 1 hour) respectively; these should
        be between -2,000 and 500.
    """
    # Re-format data to fit into our pipeline
    sunspots = pd.DataFrame(index=solar_wind_7d.index, columns=["smoothed_ssn"])
    sunspots["smoothed_ssn"] = latest_sunspot_number
    
    # Process our features and grab last 32 (timesteps) hours
    features, s = preprocess_features(
        solar_wind_7d, sunspots, scaler=scaler, subset=SOLAR_WIND_FEATURES
    )
    model_input = features[-TIMESTEPS:][XCOLS].values.reshape(
        (1, TIMESTEPS, features.shape[1])
    )
    
    # Make a prediction
    prediction_at_t0, prediction_at_t1 = model.predict(model_input)[0]

    # Optional check for unexpected values
    if not np.isfinite(prediction_at_t0):
        prediction_at_t0 = -12
    if not np.isfinite(prediction_at_t1):
        prediction_at_t1 = -12

    return prediction_at_t0, prediction_at_t1